In [2]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime

# Path to the fed_csv folder
fed_csv_path = Path("../data/fed_csv")

# List to store all processed dataframes
all_dataframes = []

# Get all CSV files
csv_files = list(fed_csv_path.glob("*.csv"))

print(f"Found {len(csv_files)} CSV files\n")

# Process each CSV file
for csv_file in csv_files:
    # Extract title from filename (remove .csv extension)
    title = csv_file.stem
    
    print(f"Processing: {title}")
    
    try:
        # Read the CSV file
        df = pd.read_csv(csv_file)
        
        # Identify date column (should be "Date (UTC)")
        date_col = None
        for col in df.columns:
            if 'date' in col.lower() and 'utc' in col.lower():
                date_col = col
                break
        
        if date_col is None:
            print(f"  ⚠ Warning: No date column found, skipping")
            continue
        
        # Identify outcome columns (all columns except Date and Timestamp)
        outcome_cols = [col for col in df.columns 
                       if col != date_col and 'timestamp' not in col.lower()]
        
        if len(outcome_cols) == 0:
            print(f"  ⚠ Warning: No outcome columns found, skipping")
            continue
        
        print(f"  Found {len(outcome_cols)} outcome columns: {outcome_cols}")
        
        # Convert date to datetime
        df[date_col] = pd.to_datetime(df[date_col], format='%m-%d-%Y %H:%M', errors='coerce')
        
        # Convert outcome columns to numeric (handling empty strings)
        for col in outcome_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Group by date and merge rows with same date
        # For each outcome column, take the first non-null value across all rows for that date
        merged_df = df.groupby(date_col).agg({
            col: lambda x: x.dropna().iloc[0] if len(x.dropna()) > 0 else None 
            for col in outcome_cols
        }).reset_index()
        
        # Add title column
        merged_df['Title'] = title
        
        # Rename date column to standard name
        merged_df = merged_df.rename(columns={date_col: 'Date'})
        
        # Reorder columns: Date, Title, then all outcome columns
        cols = ['Date', 'Title'] + outcome_cols
        merged_df = merged_df[cols]
        
        print(f"  ✓ Processed {len(merged_df)} unique dates (from {len(df)} rows)")
        
        all_dataframes.append(merged_df)
        
    except Exception as e:
        print(f"  ✗ Error processing {title}: {e}")
        continue

print(f"\n=== MERGING ALL DATAFRAMES ===")

if len(all_dataframes) == 0:
    print("No dataframes to merge!")
else:
    # Combine all dataframes
    # We need to handle different column structures
    # Strategy: Create a union of all columns, then merge
    
    # Get all unique outcome columns across all dataframes
    all_outcome_cols = set()
    for df in all_dataframes:
        all_outcome_cols.update([col for col in df.columns if col not in ['Date', 'Title']])
    
    all_outcome_cols = sorted(list(all_outcome_cols))
    
    print(f"Found {len(all_outcome_cols)} unique outcome columns across all files")
    
    # Create final dataframe
    final_dfs = []
    for df in all_dataframes:
        # Add missing columns with NaN
        for col in all_outcome_cols:
            if col not in df.columns:
                df[col] = None
        
        # Reorder columns
        cols = ['Date', 'Title'] + all_outcome_cols
        df = df[cols]
        final_dfs.append(df)
    
    # Concatenate all dataframes
    final_df = pd.concat(final_dfs, ignore_index=True)
    
    # Sort by Date and Title
    final_df = final_df.sort_values(['Date', 'Title']).reset_index(drop=True)
    
    print(f"\n=== FINAL DATAFRAME ===")
    print(f"Shape: {final_df.shape}")
    print(f"Date range: {final_df['Date'].min()} to {final_df['Date'].max()}")
    print(f"Unique titles: {final_df['Title'].nunique()}")
    print(f"\nColumns: {list(final_df.columns)}")
    print(f"\nFirst few rows:")
    print(final_df.head(20))
    print(f"\nDataframe info:")
    print(final_df.info())
    
    # Save to CSV
    output_path = "../data/fed_events_merged.csv"
    final_df.to_csv(output_path, index=False)
    print(f"\n✓ Saved merged dataframe to: {output_path}")

Found 13 CSV files

Processing: Fed abolished in 2025
  Found 1 outcome columns: ['Price']
  ✓ Processed 247 unique dates (from 247 rows)
Processing: Fed decision in December
  Found 4 outcome columns: ['50+ bps decrease', '25 bps decrease', 'No change', '25+ bps increase']
  ✓ Processed 99 unique dates (from 338 rows)
Processing: Fed decision in January
  Found 4 outcome columns: ['50+ bps decrease', '25 bps decrease', 'No change', '25+ bps increase']
  ✓ Processed 51 unique dates (from 185 rows)
Processing: Fed decision in March
  Found 4 outcome columns: ['50+ bps decrease', '25 bps decrease', 'No change', '25+ bps increase']
  ✓ Processed 9 unique dates (from 20 rows)
Processing: Fed decisions (Oct-Jan)
  Found 7 outcome columns: ['Cut–Cut–Pause', 'Cut–Cut–Cut', 'Cut–Pause–Cut', 'Cut–Pause–Pause', 'Other', 'Pause–Cut–Pause', 'Pause–Cut–Cut']
  ✓ Processed 39 unique dates (from 195 rows)
Processing: Fed decisions (Sep-Dec)
  Found 6 outcome columns: ['Cut–Cut–Cut', 'Cut–Cut–Pause', 

C:\Users\Michael\AppData\Local\Temp\ipykernel_27716\1730892751.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(final_dfs, ignore_index=True)


In [3]:
import pandas as pd
import numpy as np
import re

# Read the merged dataframe (or use final_df if it's already in memory)
df = final_df.copy() if 'final_df' in locals() else pd.read_csv("../data/fed_events_merged.csv")

print(f"Original merged dataframe shape: {df.shape}")

# Convert Date to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Convert from wide to long format
# Melt all outcome columns into rows
id_vars = ['Date', 'Title']
outcome_cols = [col for col in df.columns if col not in id_vars]

print(f"\nConverting to long format...")
print(f"Outcome columns: {len(outcome_cols)}")

# Melt the dataframe
df_long = pd.melt(
    df,
    id_vars=id_vars,
    value_vars=outcome_cols,
    var_name='Outcome',
    value_name='Probability'
)

# Remove rows where probability is NaN or empty
df_long = df_long.dropna(subset=['Probability'])

print(f"Long format shape: {df_long.shape}")
print(f"Sample long format data:")
print(df_long.head(10))

# Create feature names by combining Title and Outcome
# Sanitize for column names (remove special chars, spaces, etc.)
def sanitize_feature_name(title, outcome):
    # Combine title and outcome
    feature = f"{title}_{outcome}"
    # Remove special characters, keep only alphanumeric, underscore, and spaces
    feature = re.sub(r'[^a-zA-Z0-9_\s]', '', feature)
    # Replace spaces with underscores
    feature = re.sub(r'\s+', '_', feature)
    # Remove multiple underscores
    feature = re.sub(r'_+', '_', feature)
    # Remove leading/trailing underscores
    feature = feature.strip('_')
    return feature

print(f"\nCreating feature names...")
df_long['Feature'] = df_long.apply(
    lambda row: sanitize_feature_name(row['Title'], row['Outcome']), 
    axis=1
)

print(f"Unique features created: {df_long['Feature'].nunique()}")
print(f"\nSample features:")
print(df_long[['Title', 'Outcome', 'Feature']].head(20))

# Pivot to wide format: one row per date, one column per feature
print(f"\nPivoting to ML-ready format...")
df_ml = df_long.pivot_table(
    index='Date',
    columns='Feature',
    values='Probability',
    aggfunc='first'  # Take first value if duplicates (shouldn't happen)
).reset_index()

# Sort columns: Date first, then alphabetically
cols = ['Date'] + sorted([col for col in df_ml.columns if col != 'Date'])
df_ml = df_ml[cols]

print(f"\n=== ML-READY DATAFRAME ===")
print(f"Shape: {df_ml.shape}")
print(f"Rows (dates): {len(df_ml)}")
print(f"Features: {len(df_ml.columns) - 1}")  # -1 for Date column
print(f"Date range: {df_ml['Date'].min()} to {df_ml['Date'].max()}")

# Check for missing values
missing_pct = (df_ml.isnull().sum() / len(df_ml) * 100).sort_values(ascending=False)
print(f"\nFeatures with most missing values:")
print(missing_pct.head(10))

# Show sample
print(f"\nFirst few rows (showing first 10 columns):")
print(df_ml.iloc[:5, :10])
print(f"\nColumn names (first 20):")
print(df_ml.columns[:20].tolist())

# Save ML-ready dataframe
df_ml.to_csv("../data/fed_events_ml_ready.csv", index=False)
print(f"\n✓ Saved ML-ready dataframe to: ../data/fed_events_ml_ready.csv")

# Also create a version with forward-filled values (for time series)
print("\n=== CREATING FORWARD-FILLED VERSION ===")
df_ml_ffill = df_ml.copy()
df_ml_ffill = df_ml_ffill.sort_values('Date')
# Forward fill missing values (carry last known value forward)
df_ml_ffill.iloc[:, 1:] = df_ml_ffill.iloc[:, 1:].fillna(method='ffill')
# Fill any remaining NaN at the beginning with 0
df_ml_ffill.iloc[:, 1:] = df_ml_ffill.iloc[:, 1:].fillna(0)

df_ml_ffill.to_csv("../data/fed_events_ml_ready_ffill.csv", index=False)
print(f"✓ Saved forward-filled version to: ../data/fed_events_ml_ready_ffill.csv")

# Create summary statistics
print("\n=== SUMMARY STATISTICS ===")
print(f"Total dates: {len(df_ml)}")
print(f"Total features: {len(df_ml.columns) - 1}")
print(f"Features with data: {(df_ml.iloc[:, 1:].notna().any(axis=0)).sum()}")
print(f"Average features per date: {(df_ml.iloc[:, 1:].notna().sum(axis=1)).mean():.1f}")
print(f"Min features per date: {(df_ml.iloc[:, 1:].notna().sum(axis=1)).min()}")
print(f"Max features per date: {(df_ml.iloc[:, 1:].notna().sum(axis=1)).max()}")

# Show which events have the most features
print(f"\n=== EVENT FEATURE COUNTS ===")
event_feature_counts = df_long.groupby('Title')['Feature'].nunique().sort_values(ascending=False)
print(event_feature_counts.head(10))

Original merged dataframe shape: (2067, 58)

Converting to long format...
Outcome columns: 56
Long format shape: (10098, 4)
Sample long format data:
         Date                           Title Outcome  Probability
2  2024-12-30  How many Fed rate cuts in 2025       0        0.140
5  2024-12-31  How many Fed rate cuts in 2025       0        0.110
8  2025-01-01  How many Fed rate cuts in 2025       0        0.105
11 2025-01-02  How many Fed rate cuts in 2025       0        0.115
14 2025-01-03  How many Fed rate cuts in 2025       0        0.115
17 2025-01-04  How many Fed rate cuts in 2025       0        0.105
20 2025-01-05  How many Fed rate cuts in 2025       0        0.105
23 2025-01-06  How many Fed rate cuts in 2025       0        0.095
26 2025-01-07  How many Fed rate cuts in 2025       0        0.125
29 2025-01-08  How many Fed rate cuts in 2025       0        0.115

Creating feature names...
Unique features created: 88

Sample features:
                             Title Outcom

C:\Users\Michael\AppData\Local\Temp\ipykernel_27716\1478442450.py:101: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ml_ffill.iloc[:, 1:] = df_ml_ffill.iloc[:, 1:].fillna(method='ffill')


In [4]:
import pandas as pd
import numpy as np

# Read the datasets
print("Loading datasets...")
fed_ml = pd.read_csv("../data/fed_events_ml_ready_ffill.csv")
sp500 = pd.read_csv("../data/s&p_data/sp500_ohlcv_returns.csv")

print(f"Fed ML-ready shape: {fed_ml.shape}")
print(f"Fed date range: {fed_ml['Date'].min()} to {fed_ml['Date'].max()}")
print(f"\nS&P 500 shape: {sp500.shape}")
print(f"S&P 500 date range: {sp500['Date (UTC)'].min()} to {sp500['Date (UTC)'].max()}")

# Convert dates to datetime
fed_ml['Date'] = pd.to_datetime(fed_ml['Date'])
sp500['Date (UTC)'] = pd.to_datetime(sp500['Date (UTC)'])

# Merge on date
print("\nMerging datasets...")
merged = pd.merge(
    fed_ml,
    sp500,
    left_on='Date',
    right_on='Date (UTC)',
    how='inner'  # Keep only dates where both have data
)

# Drop the duplicate date column
merged = merged.drop('Date (UTC)', axis=1)

# Sort by date
merged = merged.sort_values('Date').reset_index(drop=True)

print(f"\nMerged dataset shape: {merged.shape}")
print(f"Merged date range: {merged['Date'].min()} to {merged['Date'].max()}")
print(f"Total columns: {len(merged.columns)}")

# Show column breakdown
fed_cols = [col for col in merged.columns if col.startswith('Fed_') or 'How_many' in col or 'Jerome' in col or 'Who_will' in col]
sp500_cols = [col for col in merged.columns if col not in fed_cols and col != 'Date']

print(f"\nColumn breakdown:")
print(f"  - Fed features: {len(fed_cols)}")
print(f"  - S&P 500 features: {len(sp500_cols)}")
print(f"  - Date column: 1")

# Show sample of merged data
print(f"\n=== SAMPLE OF MERGED DATA ===")
print(merged[['Date', 'Daily_Return', 'Close'] + fed_cols[:5]].head(10))

# Check for missing values
print(f"\n=== MISSING VALUES CHECK ===")
missing_fed = merged[fed_cols].isnull().sum().sum()
missing_sp500 = merged[sp500_cols].isnull().sum().sum()
print(f"Missing values in Fed features: {missing_fed}")
print(f"Missing values in S&P 500 features: {missing_sp500}")

# Save the merged dataset
output_path = "../data/sp500_fed_merged_ml_ready.csv"
merged.to_csv(output_path, index=False)
print(f"\n✓ Saved merged dataset to: {output_path}")

# Create target variables for ML
print("\n=== CREATING TARGET VARIABLES ===")

# Next day return (shift -1 means tomorrow's return)
merged['target_next_day_return'] = merged['Daily_Return'].shift(-1)

# Next day direction (1 = up, 0 = down)
merged['target_next_day_direction'] = (merged['target_next_day_return'] > 0).astype(int)

# Future returns (5-day, 10-day)
merged['target_5day_return'] = merged['Daily_Return'].shift(-1).rolling(window=5).sum()
merged['target_10day_return'] = merged['Daily_Return'].shift(-1).rolling(window=10).sum()

# Future directions
merged['target_5day_direction'] = (merged['target_5day_return'] > 0).astype(int)
merged['target_10day_direction'] = (merged['target_10day_return'] > 0).astype(int)

# Trading signals (Buy=1, Hold=0, Sell=-1)
return_threshold = 0.01  # 1%
merged['target_trading_signal'] = 0  # Hold
merged.loc[merged['target_next_day_return'] > return_threshold, 'target_trading_signal'] = 1  # Buy
merged.loc[merged['target_next_day_return'] < -return_threshold, 'target_trading_signal'] = -1  # Sell

print("Target variables created:")
print(f"  - target_next_day_return: {merged['target_next_day_return'].notna().sum()} values")
print(f"  - target_next_day_direction: {merged['target_next_day_direction'].notna().sum()} values")
print(f"  - target_5day_return: {merged['target_5day_return'].notna().sum()} values")
print(f"  - target_10day_return: {merged['target_10day_return'].notna().sum()} values")
print(f"  - target_trading_signal distribution: {merged['target_trading_signal'].value_counts().to_dict()}")

# Save final dataset with targets
final_output_path = "../data/sp500_fed_ml_ready_with_targets.csv"
merged.to_csv(final_output_path, index=False)
print(f"\n✓ Saved final dataset with targets to: {final_output_path}")

# Summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Total rows: {len(merged)}")
print(f"Total features: {len(fed_cols) + len(sp500_cols)}")
print(f"  - Fed probability features: {len(fed_cols)}")
print(f"  - S&P 500 technical features: {len(sp500_cols)}")
print(f"Target variables: 6")
print(f"\nReady for XGBoost training!")

Loading datasets...
Fed ML-ready shape: (320, 89)
Fed date range: 2024-12-30 00:00:00 to 2025-11-06 17:50:00

S&P 500 shape: (656, 15)
S&P 500 date range: 2023-03-15 to 2025-10-23

Merging datasets...

Merged dataset shape: (205, 103)
Merged date range: 2024-12-30 00:00:00 to 2025-10-23 00:00:00
Total columns: 103

Column breakdown:
  - Fed features: 88
  - S&P 500 features: 14
  - Date column: 1

=== SAMPLE OF MERGED DATA ===
        Date  Daily_Return        Close  Fed_abolished_in_2025_Price  \
0 2024-12-30     -0.010702  5906.939941                          0.0   
1 2024-12-31     -0.004285  5881.629883                          0.0   
2 2025-01-02     -0.002224  5868.549805                          0.0   
3 2025-01-03      0.012596  5942.470215                          0.0   
4 2025-01-06      0.005538  5975.379883                          0.0   
5 2025-01-07     -0.011104  5909.029785                          0.0   
6 2025-01-08      0.001560  5918.250000                          